In [57]:
import numpy as np
from make_data import classification_data
from collections import Counter
X_train , X_test, y_train, y_test = classification_data()

In [2]:
def entropy(y):
    hist = np.bincount(y)
    ps = hist / len(y)
    return -np.sum([p * np.log2(p) for p in ps if p > 0])

In [23]:
class Node:
    def __init__(self, feature = None , threshold = None, left = None , right = None,* , value = None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None
    

class DecisionTree:
    def __init__(self, min_sample_split=2, max_depth=100, n_feats=None):
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None

    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        self.root = self._grow_trees(X, y)

    def _grow_trees(self, X, y, depth=0):
        n_samples , n_features = X.shape
        n_labels = len(np.unique(y))

        # Stopping criteria
        if (depth >= self.max_depth 
            or n_labels == 1 
            or n_samples < self.min_sample_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

        # greedy search 
        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idx , right_idx = self._split(X[: , best_feat], best_thresh)
        left = self._grow_trees(X[left_idx, :], y[left_idx], depth + 1)
        right = self._grow_trees(X[right_idx, :], y[right_idx], depth + 1)

        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx , split_thres = None , None 
        for feat_idx in feat_idxs:
            x_colomn =  X[:, feat_idx]
            threasholds = np.unique(x_colomn)
            for threashold in threasholds :
                gain = self._information_gain(y, x_colomn, threashold)

                if gain > best_gain:
                    best_gain = gain 
                    split_idx = feat_idx
                    split_thres = threashold
        
        return split_idx , split_thres
    
    def _information_gain(self,y, x_coloumn, split_threh):
        # Parent entropy
        parent_entropy = entropy(y)

        # Generate split
        left_idx, right_idx = self._split(x_coloumn, split_threh)
        left_len , right_len = len(left_idx), len(right_idx)
        if left_len == 0 or right_len == 0: return 0

        # weighted average child 
        n = len(y)
        e_l , e_r = entropy(y[left_idx]), entropy(y[right_idx]) 
        child_entropy = (left_len / n)*e_l + (right_len / n)*e_r

        return parent_entropy - child_entropy

    def _split(self, x_coloum, split_threh):
        left_idx = np.argwhere(x_coloum <= split_threh).flatten()
        right_idx = np.argwhere(x_coloum > split_threh).flatten()
        return left_idx, right_idx

    def predict(self, X):
        return np.array([self._traverse_tree(x,self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        
        return self._traverse_tree(x, node.right)

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

In [24]:
model = DecisionTree()

In [25]:
model.fit(X_train, y_train)

In [27]:
preds = model.predict(X_test)

In [28]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [29]:
accuracy(y_test, preds)

np.float64(0.97)

In [52]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idx = np.random.choice(n_samples, size=n_samples, replace=True)
    return X[idx], y[idx]

In [83]:
class RandomForest():
    def __init__(self, n_trees= 100, min_sample_split=2, max_depth=150, n_feats=None):
        self.n_trees = n_trees
        self.min_sample_split = min_sample_split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.trees = []
    def fit(self, X, y):
        for _ in range(self.n_trees):
            tree = DecisionTree(min_sample_split=self.min_sample_split,
                                max_depth=self.max_depth,
                                n_feats=self.n_feats)
            
            x_sample , y_sample = bootstrap_sample(X_train, y_train)

            tree.fit(x_sample, y_sample)

            self.trees.append(tree)

    def predict(self, X):
        tree_preds = [tree.predict(X) for tree in self.trees]
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_preds = [self.most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_preds)
    
    def most_common_label(self, preds):
        counter = Counter(preds)
        return counter.most_common(1)[0][0]

In [84]:
model = RandomForest()

In [85]:
model.fit(X_train, y_train)

In [86]:
preds = model.predict(X_test)

In [87]:
accuracy(y_test, preds)

np.float64(0.995)